# State Estimation

State estimation (SE) determines the most likely operating state of a power system from a set of noisy, redundant measurements. While power flow computes the exact solution given precise inputs, state estimation works with real-world data that is inherently imprecise. It processes measurements such as bus voltage magnitudes, power injections, and branch flows, and produces the best estimate of all bus voltage magnitudes and angles in a least-squares sense.

State estimation is essential for real-time monitoring and control in energy management systems. The weighted least squares (WLS) method, which ANDES implements, minimizes the weighted sum of squared measurement residuals. Redundancy in measurements provides both robustness to individual measurement errors and the ability to detect bad data through statistical tests.

This tutorial covers running state estimation in ANDES, constructing custom measurement sets, interpreting results, performing bad data detection with the chi-squared test, and handling multi-island systems.

In [ ]:
# Reduce logging verbosity for PDF builds
import os
if os.environ.get('SPHINX_BUILD_PDF'):
    import andes
    _orig_config_logger = andes.config_logger
    def _quiet_logger(stream_level=20, *args, **kwargs):
        stream_level = max(stream_level, 30)
        return _orig_config_logger(stream_level, *args, **kwargs)
    andes.config_logger = _quiet_logger

## Setup

We begin by importing ANDES, NumPy, and the measurement framework. State estimation requires a converged power flow as the starting point, so we load a case and run power flow first.

In [ ]:
%matplotlib inline

import numpy as np
import andes
from andes.se import Measurements

andes.config_logger(stream_level=20)

In [ ]:
ss = andes.load(andes.get_case('ieee14/ieee14.json'))
ss.PFlow.run()

## Running State Estimation

The simplest way to run state estimation is to call `ss.SE.run()` with no arguments. ANDES will automatically generate a default set of measurements from the power flow solution, add Gaussian noise, and estimate the system state using the WLS algorithm.

In [ ]:
ss.SE.run()

The SE report shows convergence status, the number of iterations, the objective function value $J$, the number of measurements and states, and the measurement redundancy ratio. The estimation error (Max |dV| and Max |da|) compares the estimated state against the power flow solution.

After convergence, the estimated bus voltage magnitudes and angles are available through the `v_est` and `a_est` properties.

In [ ]:
print(f"Converged: {ss.SE.converged}")
print(f"Estimated voltages (pu): {np.round(ss.SE.v_est, 4)}")
print(f"Estimated angles (rad):  {np.round(ss.SE.a_est, 4)}")

## Custom Measurements

In practice, not every bus and branch is instrumented. The `Measurements` class allows you to construct a custom measurement set that reflects the actual placement of meters in the system. Each measurement is characterized by its type, location, and standard deviation (sigma), which represents the expected measurement noise.

Four types of measurements are supported:

| Method | Measurement type | Description |
|--------|-----------------|-------------|
| `add_bus_voltage()` | Voltage magnitude | RTU or PMU voltage measurements |
| `add_bus_angle()` | Voltage angle | PMU angle measurements |
| `add_bus_injection()` | P and Q injection | Bus power injection measurements |
| `add_line_flow()` | P and Q flow | Branch power flow measurements (from-end) |

The standard deviation sigma controls the weight of each measurement in the WLS objective. A smaller sigma means the estimator trusts that measurement more. Typical values are 0.005-0.02 for voltage measurements and 0.02-0.05 for power measurements.

In [ ]:
m = Measurements(ss)

# Voltage meters at all buses (high accuracy)
m.add_bus_voltage(sigma=0.01)

# Power injection meters at all buses
m.add_bus_injection(sigma_p=0.02, sigma_q=0.03)

# Generate noisy measurement values from the PFlow solution
m.generate_from_pflow(seed=42)

print(f"Total measurements: {m.nm}")
print(f"State dimension: {2 * ss.Bus.n}")
print(f"Redundancy: {m.nm / (2 * ss.Bus.n):.1f}x")

The redundancy ratio (measurements divided by states) indicates how much extra information is available. A ratio above 1.0 means the system is overdetermined, which is necessary for state estimation to work. Higher redundancy provides better noise rejection and more robust bad data detection. A redundancy of 2-3x is typical for real systems.

Pass the custom measurements to `SE.run()` to use them instead of the default set.

In [ ]:
ss.SE.run(measurements=m)

### Selective Meter Placement

You can also place measurements on specific buses or lines rather than the entire system. This is useful for modeling realistic SCADA configurations where only key substations have full instrumentation. The `bus_idx` and `line_idx` parameters accept a list of device indices.

In [ ]:
m2 = Measurements(ss)

# Voltage at selected buses only
m2.add_bus_voltage(bus_idx=[1, 2, 3, 6, 8], sigma=0.005)

# PMU angle measurement at the slack bus
m2.add_bus_angle(bus_idx=[1], sigma=0.01)

# Injection meters at all buses
m2.add_bus_injection(sigma_p=0.02, sigma_q=0.03)

# Flow meters on selected lines
m2.add_line_flow(line_idx=['Line_1', 'Line_2', 'Line_3', 'Line_7', 'Line_8'],
                 sigma_p=0.02, sigma_q=0.03)

m2.generate_from_pflow(seed=42)

print(f"Total measurements: {m2.nm}")
ss.SE.run(measurements=m2)

### Low-Level `add()` Method

All convenience wrappers delegate to the `add()` method, which accepts a model name and variable name. This method is useful when you need fine-grained control over individual measurements.

In [ ]:
m3 = Measurements(ss)

# Equivalent to m3.add_bus_voltage(bus_idx=[1, 2], sigma=0.005)
m3.add('Bus', 'v', idx=[1, 2], sigma=0.005)

# Equivalent to m3.add_bus_angle(bus_idx=[1], sigma=0.01)
m3.add('Bus', 'a', idx=[1], sigma=0.01)

print(f"Measurements added: {m3.nm}")

## Zero-Noise Verification

A useful sanity check is to verify that state estimation recovers the exact power flow solution when given perfect (noise-free) measurements. In this case, the measurement vector is set to $z = h(x_{\text{true}})$ without any added noise. The WLS estimator should converge to the true state to within numerical precision.

This test validates the correctness of the measurement functions $h(x)$ and the Jacobian $H(x)$.

In [ ]:
from andes.se import StaticEvaluator

m_exact = Measurements(ss)
m_exact.add_bus_voltage(sigma=0.01)
m_exact.add_bus_angle(sigma=0.01)
m_exact.add_bus_injection(sigma_p=0.02, sigma_q=0.03)
m_exact.add_line_flow(sigma_p=0.02, sigma_q=0.03)

# Set z = h(x_true) exactly (no noise)
m_exact.finalize()
ev = StaticEvaluator(ss, m_exact)
theta_true = np.array(ss.Bus.a.v, dtype=float)
Vm_true = np.array(ss.Bus.v.v, dtype=float)
m_exact.z = ev.h(theta_true, Vm_true)

ss.SE.run(measurements=m_exact)

v_err = np.max(np.abs(ss.SE.v_est - Vm_true))
a_err = np.max(np.abs(ss.SE.a_est - theta_true))
print(f"Max voltage error: {v_err:.2e} pu")
print(f"Max angle error:   {a_err:.2e} rad")

The errors are at the level of machine precision ($\sim 10^{-11}$), confirming that the measurement model and solver are implemented correctly.

## Bad Data Detection

The chi-squared ($\chi^2$) test is a standard method for detecting the presence of bad data in the measurement set. Under the assumption that measurement errors are Gaussian, the weighted sum of squared residuals $J(\hat{x}) = [z - h(\hat{x})]^T W [z - h(\hat{x})]$ follows a chi-squared distribution with $m - n$ degrees of freedom, where $m$ is the number of measurements and $n$ is the number of states.

If $J$ exceeds the chi-squared threshold at the desired confidence level, the test rejects the null hypothesis that all measurements are normal, indicating possible bad data.

In [ ]:
# Run SE with normal measurements
m_normal = Measurements(ss)
m_normal.add_bus_voltage(sigma=0.01)
m_normal.add_bus_injection(sigma_p=0.02, sigma_q=0.03)
m_normal.generate_from_pflow(seed=42)

ss.SE.run(measurements=m_normal)

passed, J, threshold, dof = ss.SE.chi_squared_test()
print(f"Chi-squared test: {'PASSED' if passed else 'FAILED'}")
print(f"  J = {J:.4f}, threshold = {threshold:.4f}, dof = {dof}")

Now we inject a gross error into one measurement (a 10-sigma outlier) and observe the effect on the chi-squared test.

In [ ]:
# Inject a gross error
m_bad = Measurements(ss)
m_bad.add_bus_voltage(sigma=0.01)
m_bad.add_bus_injection(sigma_p=0.02, sigma_q=0.03)
m_bad.generate_from_pflow(seed=42)

# Corrupt one voltage measurement with a 10-sigma outlier
m_bad.z[0] += 10 * m_bad.sigma[0]

ss.SE.run(measurements=m_bad)

passed, J, threshold, dof = ss.SE.chi_squared_test()
print(f"Chi-squared test: {'PASSED' if passed else 'FAILED'}")
print(f"  J = {J:.4f}, threshold = {threshold:.4f}, dof = {dof}")

The chi-squared test correctly detects the presence of bad data. In practice, after detection, further analysis (such as the largest normalized residual test) would be used to identify and remove the specific bad measurement.

## Multi-Island Systems

In systems with multiple electrical islands (disconnected components), each island requires its own voltage angle reference because absolute angles are not observable from power measurements alone. ANDES handles this automatically by detecting islands through connectivity analysis and adding a pseudo-measurement for the angle reference at the slack bus of each island.

The Kundur two-area system provides a convenient example when configured as two separate islands.

In [ ]:
ss_island = andes.load(andes.get_case('kundur/kundur_islands.xlsx'),
                       default_config=True)
ss_island.PFlow.run()

print(f"Number of islands: {len(ss_island.Bus.island_sets)}")

In [ ]:
m_island = Measurements(ss_island)
m_island.add_bus_voltage(sigma=0.01)
m_island.add_bus_injection(sigma_p=0.02, sigma_q=0.03)
m_island.generate_from_pflow(seed=42)

ss_island.SE.run(measurements=m_island)

The log shows that ANDES added angle references for both islands automatically. Without these references, the WLS gain matrix would be singular and the estimator would fail.

## Configuration

The SE routine provides several configuration options accessible through `ss.SE.config`.

In [ ]:
ss.SE.config

| Option | Default | Description |
|--------|---------|-------------|
| `tol` | 1e-4 | Convergence tolerance for the WLS iteration |
| `max_iter` | 20 | Maximum number of Gauss-Newton iterations |
| `flat_start` | 0 | Use flat start (1.0 pu, 0 rad) instead of PFlow solution |
| `report` | 1 | Print result summary after estimation |

The flat start option is useful for testing robustness of convergence when the power flow solution is not available as an initial guess.

In [ ]:
# Run with flat start
ss.SE.config.flat_start = 1
ss.SE.run(measurements=m_normal)

# Reset to default
ss.SE.config.flat_start = 0

## Cleanup

In [ ]:
!andes misc -C

## Next Steps

- {doc}`07-eigenvalue-analysis` - Small-signal stability assessment
- {doc}`09-contingency-analysis` - N-1 contingency screening